In [2]:
import pandas as pd
import numpy as np
import sympy
import datetime as dt
import time
from math import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from ipyleaflet import *
import folium
import json
import geopy.distance
from haversine import haversine
from tqdm import tqdm_notebook

sns.set()
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

from matplotlib import font_manager, rc

import matplotlib.font_manager as fm
font_location = "C:\Windows\Fonts\malgun.ttf"
font_name = fm.FontProperties(fname=font_location).get_name()
mpl.rc("font", family=font_name)

C:\Users\LG\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
taxi = pd.read_csv("train.csv")

In [4]:
taxi.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [5]:
taxi.describe().round(2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,1.66,-73.97,40.75,-73.97,40.75,959.49
std,0.50,1.31,0.07,0.03,0.07,0.04,5237.43
min,1.00,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,9.00,-61.34,51.88,-61.34,43.92,3526282.00


- NaN값 유무
    - 어떠한 column도 NaN값을 가지지 않음.
- 분포의 대칭 여부
    - pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude : mean값과 중앙값이 거의 일치하므로 대칭으로 보임.
    - vendor_id, passenger_count : mean값과 중앙값이 많이 차이는 안나지만 대칭은 아닌것으로 보임.
    - trip_duration : mean값과 중앙값이 차이가 많이 나는것으로 보아 대칭이 아닌것으로 보임.    

In [6]:
pickup_datetime_dt = pd.to_datetime(taxi["pickup_datetime"])
dropoff_datetime_dt = pd.to_datetime(taxi["dropoff_datetime"])

In [7]:
taxi_df1 = taxi.loc[:, ["trip_duration"]]
taxi_df1["pickup_datetime"] = pickup_datetime_dt

In [8]:
# pickup datetime 중 year, month, day
taxi_df1.loc[:, "pickup_date"] = taxi_df1["pickup_datetime"].dt.date
# pickup datetime 중 month만 가져와서 새로 column을 만듦.
taxi_df1.loc[:, "pickup_month"] = taxi_df1["pickup_datetime"].dt.month
# pickup datetime 중 hour만 가져와서 새로 column을 만듦.
taxi_df1.loc[:, "pickup_hour"] = taxi_df1["pickup_datetime"].dt.hour
# pickup datetime 중 요일만 가져와서 새로 column을 만듦.
# 월요일은 0이고, 일요일은 6임.
taxi_df1.loc[:, "pickup_weekday"] = taxi_df1["pickup_datetime"].dt.weekday

In [9]:
taxi_df1.tail()

,trip_duration,pickup_datetime,pickup_date,pickup_month,pickup_hour,pickup_weekday
1458639,778,2016-04-08 13:31:04,2016-04-08,4,13,4
1458640,655,2016-01-10 07:35:15,2016-01-10,1,7,6
1458641,764,2016-04-22 06:57:41,2016-04-22,4,6,4
1458642,373,2016-01-05 15:56:26,2016-01-05,1,15,1
1458643,198,2016-04-05 14:44:25,2016-04-05,4,14,1


In [10]:
taxi_df1.describe().round(2)

,trip_duration,pickup_month,pickup_hour,pickup_weekday
count,1458644.00,1458644.00,1458644.00,1458644.00
mean,959.49,3.52,13.61,3.05
std,5237.43,1.68,6.40,1.95
min,1.00,1.00,0.00,0.00
25%,397.00,2.00,9.00,1.00
50%,662.00,4.00,14.00,3.00
75%,1075.00,5.00,19.00,5.00
max,3526282.00,6.00,23.00,6.00


In [11]:
taxi_df2 = taxi_df1.drop(labels = ["pickup_datetime", "pickup_date"], axis = 1)

In [12]:
taxi_df2.tail()

,trip_duration,pickup_month,pickup_hour,pickup_weekday
1458639,778,4,13,4
1458640,655,1,7,6
1458641,764,4,6,4
1458642,373,1,15,1
1458643,198,4,14,1


In [13]:
taxi_df2.describe().round(2)

,trip_duration,pickup_month,pickup_hour,pickup_weekday
count,1458644.00,1458644.00,1458644.00,1458644.00
mean,959.49,3.52,13.61,3.05
std,5237.43,1.68,6.40,1.95
min,1.00,1.00,0.00,0.00
25%,397.00,2.00,9.00,1.00
50%,662.00,4.00,14.00,3.00
75%,1075.00,5.00,19.00,5.00
max,3526282.00,6.00,23.00,6.00


In [14]:
dfX0 = pd.DataFrame(taxi_df2, columns = ["pickup_month", "pickup_hour", "pickup_weekday"])
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(taxi_df2, columns = ["trip_duration"])

model_taxi = sm.OLS(dfy, dfX)
result_taxi = model_taxi.fit()
print(result_taxi.summary())

                            OLS Regression Results                            
Dep. Variable:          trip_duration   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     27.97
Date:                Thu, 08 Mar 2018   Prob (F-statistic):           4.43e-18
Time:                        19:38:23   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458640   BIC:                         2.912e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            848.4005     15.688     54.

In [15]:
model_taxi2 = sm.OLS.from_formula("trip_duration ~ C(pickup_month) - 1", taxi_df2)
result_taxi2 = model_taxi2.fit()
sm.stats.anova_lm(model_taxi2.fit())

C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(pickup_month),6.0,1.344743e+12,2.241238e+11,8170.903252,0.0
Residual,1458638.0,4.000971e+13,2.742950e+07,NaN,NaN


In [18]:
print(result_taxi2.summary())

                            OLS Regression Results                            
Dep. Variable:          trip_duration   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     13.69
Date:                Thu, 08 Mar 2018   Prob (F-statistic):           2.14e-13
Time:                        19:40:02   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458638   BIC:                         2.912e+07
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(pickup_month)[1]   922.3733     10

In [16]:
model_taxi3 = sm.OLS.from_formula("trip_duration ~ C(pickup_hour) - 1", taxi_df2)
result_taxi3 = model_taxi3.fit()
sm.stats.anova_lm(model_taxi3.fit())

C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(pickup_hour),24.0,1.351902e+12,5.632923e+10,2053.942845,0.0
Residual,1458620.0,4.000255e+13,2.742493e+07,NaN,NaN


In [19]:
print(result_taxi3.summary())

                            OLS Regression Results                            
Dep. Variable:          trip_duration   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     14.33
Date:                Thu, 08 Mar 2018   Prob (F-statistic):           4.83e-56
Time:                        19:40:12   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458620   BIC:                         2.912e+07
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(pickup_hour)[0]    936.6573     22

In [17]:
model_taxi4 = sm.OLS.from_formula("trip_duration ~ C(pickup_weekday) - 1", taxi_df2)
result_taxi4 = model_taxi4.fit()
sm.stats.anova_lm(model_taxi4.fit())

C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(pickup_weekday),7.0,1.345122e+12,1.921603e+11,7005.669525,0.0
Residual,1458637.0,4.000933e+13,2.742926e+07,NaN,NaN


In [20]:
print(result_taxi4.summary())

                            OLS Regression Results                            
Dep. Variable:          trip_duration   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     13.72
Date:                Thu, 08 Mar 2018   Prob (F-statistic):           1.20e-15
Time:                        19:40:20   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458637   BIC:                         2.912e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
C(pickup_weekday)[0]   897.9478 

In [15]:
taxi_df2.shape

(1458644, 4)

In [16]:
taxi_df2.tail()

,trip_duration,pickup_month,pickup_hour,pickup_weekday
1458639,778,4,13,4
1458640,655,1,7,6
1458641,764,4,6,4
1458642,373,1,15,1
1458643,198,4,14,1


In [22]:
pickup_weekend = []
for i in range(1458644):
    if taxi_df2.iat[i, 3] == 4:
        pickup_weekend.append("Yes")
    elif taxi_df2.iat[i, 3] == 5:
        pickup_weekend.append("Yes")
    elif taxi_df2.iat[i, 3] == 6:
        pickup_weekend.append("Yes")
    else:
        pickup_weekend.append("No")
    
taxi_df2["pickup_weekend"] = pickup_weekend
taxi_df2.tail()

,trip_duration,pickup_month,pickup_hour,pickup_weekday,pickup_weekend
1458639,778,4,13,4,Yes
1458640,655,1,7,6,Yes
1458641,764,4,6,4,Yes
1458642,373,1,15,1,No
1458643,198,4,14,1,No


In [23]:
model_taxi5 = sm.OLS.from_formula("trip_duration ~ pickup_weekend - 1", taxi_df2)
result_taxi5 = model_taxi3.fit()
sm.stats.anova_lm(model_taxi5.fit())

C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\LG\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
pickup_weekend,2.0,1.343000e+12,6.714998e+11,24479.937798,0.0
Residual,1458642.0,4.001145e+13,2.743062e+07,NaN,NaN


In [24]:
print(result_taxi5.summary())

                            OLS Regression Results                            
Dep. Variable:          trip_duration   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     14.33
Date:                Thu, 08 Mar 2018   Prob (F-statistic):           4.83e-56
Time:                        19:47:07   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458620   BIC:                         2.912e+07
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(pickup_hour)[0]    936.6573     22